In [ ]:
#Load required packages
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
#Extract csv files and convert to pandas dataframe
players_file = '../resources/players.csv'
players_df = pd.read_csv(players_file)
players_df.head()

In [ ]:
#Rename columns in dataframe and display results
players_df.rename(columns = {'playerId':'player_id', 'nameFirst':'first_name', 'nameLast':'last_name', 'nameFull':'full_name',
                              'nflId':'nfl_id', 'heightInches': 'height_inches', 
                             'ageAtDraft': 'draft_age', 'playerProfileURL': 'player_profile',
                            'homeCity': 'hometown', 'homeState': 'home_state', 'homeCountry': 'country'}, inplace = True)

players_df

In [ ]:
#Remove multiple decimal places from draft age and display results
player_values = players_df.round(decimals=0) 
player_values

In [ ]:
#Drop columns from dataframe and display results
players_drop_col = player_values.drop(['first_name', 'last_name', 'collegeId', 'combineId', 'highSchool', 'hsCity', 'hsState', 'hsCountry'], axis=1)
players_drop_col

In [ ]:
#Remove null values from columns and rows of dataframe and display results
players_drop_nulls = players_drop_col.dropna(how='any')
players_drop_nulls

In [ ]:
#Extract csv files and convert to pandas dataframe
draft_file = '../resources/draft.csv'
draft_df = pd.read_csv(draft_file)
draft_df.head()

In [ ]:
#Rename columns in dataframe and display results
draft_df.rename(columns = {'playerId':'player_id', 'nameFirst':'first_name', 'nameLast':'last_name', 'nameFull': 'full_name',
                              'teamId':'team_id', 'heightInches': 'height_inches', 
                             'ageAtDraft': 'draft_age', 'playerProfileURL': 'player_profile',
                            'homeCity': 'hometown', 'homeState': 'home_state', 'homeCountry': 'country'}, inplace = True)

draft_df

In [ ]:
#Remove six decimal places for draft age and display results
draft_values = draft_df.round(decimals=0)
draft_values

In [ ]:
#Drop columns from dataframe and display results
draft_drop_col = draft_values.drop(columns =['first_name', 'last_name', 'collegeId', 'nflId','combineId', 'highSchool', 'hsCity', 'hsState', 'hsCountry'])
draft_drop_col

In [ ]:
#drop null values from rows and columns of data frame and display results
draft_drop_nulls = draft_drop_col.dropna(how='any')
draft_drop_nulls

In [ ]:
#Extract csv and convert to pandas dataframe
receiver_file = '../resources/receiver.csv'
receiver_df = pd.read_csv(receiver_file)
receiver_df.head()

In [ ]:
df.rename(columns = {'playerId':'player_id', 'nameFirst':'first_name', 'nameLast':'last_name', 'nameFull': 'full_name',
                              'teamId':'team_id', 'heightInches': 'height_inches', 
                             'ageAtDraft': 'draft_age', 'playerProfileURL': 'player_profile',
                            'homeCity': 'hometown', 'homeState': 'home_state', 'homeCountry': 'country'}, inplace = True)

df

In [ ]:
#Create connection to sql database
connection_db = "postgres:postgres@localhost:5432/nfl_data"
engine = create_engine(f'postgresql://{connection_db}')

In [ ]:
#Display table names from postgres database
engine.table_names()

In [ ]:
#Make column values lowercase and upload pandas dataframe to sql database
draft_drop_nulls.columns = [c.lower() for c in draft_drop_nulls.columns]
draft_drop_nulls.to_sql(name='draft', con=engine, if_exists='append', index=False)


In [ ]:
#Connect to draft table from sql database and return first five rows that have been loaded
pd.read_sql_query('select * from draft', con=engine).head()

In [ ]:
#Make column values lowercase and upload pandas dataframe to sql database
players_drop_nulls.columns = [c.lower() for c in players_drop_nulls.columns]
players_drop_nulls.to_sql(name='players', con=engine, if_exists='append', index=False)

In [ ]:
#Connect to draft table from sql database and return first five rows that have been loaded
pd.read_sql_query('select * from players', con=engine).head()

## Rushing csv

In [ ]:
csv_file= "../resources/rusher.csv"
rusher_df=pd.read_csv(csv_file)
rusher_df

In [ ]:
#Dropping columns 
drop_rusher_df=rusher_df.drop(columns=['rushType', 'rushDirection', 'rushLandmark','rushEnd', 'rushNull','rushPrimary','playId'])

In [ ]:
#Renaming columns 
new_rusher_df=drop_rusher_df.rename(columns={"rushId": 'rush_id', 'teamId': 'team_id','playerId':'player_id', 
                          'rushPosition': 'rush_position', 'rushYards': 'rush_yards','rushTd':'rush_td'}).head()

In [ ]:
new_rusher_df

## Loading the receiver.csv

In [ ]:
rec_csv_file= "../resources/receiver.csv"
receiver_df=pd.read_csv(rec_csv_file)
receiver_df

In [ ]:
#Selecting the columns 
new_receiver_df = receiver_df[['receiverId','teamId', 'playerId', 'recPosition', 'recYards']].copy()
new_receiver_df.head()

In [ ]:
#Renaming columns 
name_receiver_df=new_receiver_df.rename(columns={'receiverId': 'receiver_id', 'teamId': 'team_id',
                                                'playerId': 'player_id', 'recPosition': 'rec_position',
                                               'recYards': 'rec_yards'})
name_receiver_df

##Conecting to the local data base

In [ ]:
rds_connection_string = "postgres:postgres@localhost:5432/nfl_data"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

## Loading the clean rusher.csv to database

In [ ]:
new_rusher_df.to_sql(name='rusher', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select* from rusher', con=engine).head()

Merging the receiver data

In [ ]:
receiver_data=pd.read_sql_query('select players.player_id, receiver.player_id, draft.full_name, receiver.rec_yards, receiver.rec_position from player inner join receiver on player.player_id= receiver.player_id inner join draft on player.player_id= draft.player_id'
, con=engine).head()
receiver_data

In [ ]:
receiver_data_df=receiver_data.drop(columns=['playerid'])
receiver_data_df.head()

### Merging the rush data 

In [ ]:
rush_data=pd.read_sql_query('select player.player_id, rusher.player_id, draft.full_name, rusher.rush_yards, rusher.rush_position from player inner join rusher on player.player_id= rusher.player_id inner join draft on player.player_id= draft.player_id'
, con=engine)
rush_data.head()

## Load NFL Combine Data

In [ ]:
combine_csv= "../resources/combine.csv"
combine_df=pd.read_csv(combine_csv)
combine_df

In [ ]:
#Selecting the columns 
filter_combine_df = combine_df[['playerId', 'combineHand', 'combineBench', 'combine40yd']].copy()
filter_combine_df.head()

In [ ]:
#Rename the columns
clean_combine_df=filter_combine_df.rename(columns={'playerId': 'player_id', 'combineHand': 'hand_size',
                                               'combineBench': 'bench_press_rep', 'combine40yd': 'forty_yd_time'})
clean_combine_df

## Load NFL Passer Data

In [ ]:
passer_csv= "../resources/passer.csv"
passer_df=pd.read_csv(passer_csv)
passer_df

In [ ]:
#Selecting the columns 
filter_passer_df = passer_df[['playerId', 'passId', 'teamId', 'passPosition', 'passOutcomes', 'passLength', 'passAtt', 'passComp', 'passTd', 'passInt']].copy()
filter_passer_df.head()

In [ ]:
#Rename the columns
clean_passer_df=filter_passer_df.rename(columns={'playerId': 'player_id','teamId': 'team_id','passId': 'pass_id','passPosition': 'pass_position','passOutcomes': 'pass_outcomes','passLength':'pass_length','passAtt':'pass_attempt','passComp':'pass_completion','passTd':'pass_touchdown','passInt':'pass_interception'})
clean_passer_df

## Merge Players with Combine and Passer Data

In [ ]:
new_combine_df.to_sql(name='combine', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select* from combine', con=engine).head()

In [ ]:
combine_data=pd.read_sql_query('select players.player_id, combine.player_id, draft.full_name, combine.hand_size, combine.hand_size, combine.forty_yd_time from players inner join combine on players.player_id= combine.player_id inner join draft on players.player_id= draft.player_id'
, con=engine).head()
combine_data

In [ ]:
passer_data=pd.read_sql_query('select players.player_id, passer.player_id, draft.full_name, passer.pass_outcomes, passer.pass_attempt, passer.pass_completion, passer.pass_touchdown from players inner join passer on players.player_id= passer.player_id inner join draft on players.player_id= draft.player_id'
, con=engine).head()
passer_data